# Experiments and Demonstration Record of Training

In [1]:
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB2, EfficientNetB4, EfficientNetB7
from tensorflow.keras.layers import Concatenate, Dense, Dropout, Flatten, GlobalAveragePooling2D, Input, LSTM, TimeDistributed
from tensorflow.keras import Model
import tensorflow as tf

In [2]:
from pathlib import Path
import json
import pandas as pd

In [3]:
from data.img_seq_generator import ImageSequenceDataGenerator

In [4]:
import os
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
# tf.config.set_logical_device_configuration(physical_devices[0], [tf.config.LogicalDeviceConfiguration(memory_limit=4096)])

In [5]:
OUTPUT_PATH = Path(r'D:\Uni\Honours\Project\data\autsl\outputs\runs')
INPUT_TRAIN = Path(r'D:\Uni\Honours\Project\data\autsl\frames_rgb_20\train')
INPUT_VAL = Path(r'D:\Uni\Honours\Project\data\autsl\frames_rgb_20\val')
INPUT_TEST = Path(r'D:\Uni\Honours\Project\data\autsl\frames_rgb_20\test')
INPUT_TRAIN_DEPTH = Path(r'D:\Uni\Honours\Project\data\autsl\frames_depth_20\train')
INPUT_VAL_DEPTH = Path(r'D:\Uni\Honours\Project\data\autsl\frames_depth_20\val')
INPUT_TEST_DEPTH = Path(r'D:\Uni\Honours\Project\data\autsl\frames_depth_20\test')

TRAIN_LABEL_PATH = Path(r'D:\Uni\Honours\Project\data\autsl\train_labels_20classes.csv')
VAL_LABEL_PATH = Path(r'D:\Uni\Honours\Project\data\autsl\val_labels_20classes.csv')
TEST_LABEL_PATH = Path(r'D:\Uni\Honours\Project\data\autsl\test_labels_20classes.csv')

In [6]:
train_df = pd.read_csv(TRAIN_LABEL_PATH)
val_df = pd.read_csv(VAL_LABEL_PATH)
test_df = pd.read_csv(TEST_LABEL_PATH)

In [7]:
INPUT_SHAPE=(224,224,3)
TIME_DIST_SHAPE=(None,224,224,3)
NUM_CLASSES = 20

In [8]:
train_generator = ImageSequenceDataGenerator(train_df, INPUT_TRAIN, batch_size=8, input_size=INPUT_SHAPE, shuffle=True)
val_generator = ImageSequenceDataGenerator(val_df, INPUT_VAL, batch_size=8, input_size=INPUT_SHAPE, shuffle=False)
# test_generator = ImageSequenceDataGenerator(test_df, INPUT_TEST, batch_size=8, input_size=INPUT_SHAPE, shuffle=False)

In [9]:
train_depth_generator = ImageSequenceDataGenerator(train_df, INPUT_TRAIN_DEPTH, batch_size=8, input_size=INPUT_SHAPE, shuffle=True)
val_depth_generator = ImageSequenceDataGenerator(val_df, INPUT_VAL_DEPTH, batch_size=8, input_size=INPUT_SHAPE, shuffle=False)
# test_depth_generator = ImageSequenceDataGenerator(test_df, INPUT_TEST_DEPTH, batch_size=8, input_size=INPUT_SHAPE, shuffle=False)

In [10]:
def gen_train_generator():
    for i in range(len(train_generator)):
        yield train_generator.getitem(i)
        
def gen_val_generator():
    for i in range(len(val_generator)):
        yield val_generator.getitem(i)
        
train_dataset = tf.data.Dataset.from_generator(
    gen_train_generator, 
    output_signature=(
        tf.TensorSpec(shape=(None, *TIME_DIST_SHAPE), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 20), dtype=tf.float32)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    gen_val_generator, 
    output_signature=(
        tf.TensorSpec(shape=(None, *TIME_DIST_SHAPE), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 20), dtype=tf.float32)
    )
)

In [11]:
def gen_train_depth_generator():
    for i in range(len(train_depth_generator)):
        yield train_depth_generator.getitem(i)
        
def gen_val_depth_generator():
    for i in range(len(val_depth_generator)):
        yield val_generator.getitem(i)
        
train_depth_dataset = tf.data.Dataset.from_generator(
    gen_train_depth_generator, 
    output_signature=(
        tf.TensorSpec(shape=(None, *TIME_DIST_SHAPE), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 20), dtype=tf.float32)
    )
)

val_depth_dataset = tf.data.Dataset.from_generator(
    gen_val_depth_generator, 
    output_signature=(
        tf.TensorSpec(shape=(None, *TIME_DIST_SHAPE), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 20), dtype=tf.float32)
    )
)

In [ ]:
# Generator Test
# Note that Y is ordered alphabetically and one hot encoded
train_batch0 = train_generator[0]
print("X: ", train_batch0[0][0])
print("Y: ", train_batch0[1])

In [ ]:
# Generator Test
# Note that Y is ordered alphabetically and one hot encoded
val_batch0 = val_generator[0]
print("X: ", val_batch0[0][0])
print("Y: ", val_batch0[1])

In [ ]:
# Generator Test
# Note that Y is ordered alphabetically and one hot encoded
test_batch0 = test_generator[0]
print("X: ", test_batch0[0][0])
print("Y: ", test_batch0[1])

## EfficientNetB0
https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet/EfficientNetB0

Using transfer learning, we test training with EfficientNetB0 and our custom data generator based on dataframes.

EfficientNetB0 is lightweight, and would be similar to mobile deployed platforms and can be considered an equivalent test for lightweight models based on parameter size.

EfficientNetB0 has 5,330,571 parameters, and a 29MB model size.
MobileNetV2 has 3,538,984 parameters, and a 14MB model size.
NasNetMobile has 5,326,716 parameters, and a 23MB model size.

Furthermore EfficientNetB0 scored 77.1% Top-1 Accuracy and 93.3% Top-5 Accuracy on imagenet beating ResNet50 (76.0%, 93.0%) whilst being 5x smaller.

See: https://arxiv.org/pdf/1905.11946.pdf

Therefore, I use EfficientNetB0 has the baseline transfer learning model. Other models to be used are B2 (9.2M parameters) B4 (19m parameters), and B7 (66m parameters) to gauge different sized models.

In [12]:
def get_efficientnet(model_num=0, input_size=(224,224,3), finetune=False, tune_layers=3):
    model = None
    if model_num == 0:
        model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_size)
    elif model_num == 2:
        model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=input_size)
    elif model_num == 4:
        model = EfficientNetB4(include_top=False, weights='imagenet', input_shape=input_size)
    elif model_num == 7:
        model = EfficientNetB7(include_top=False, weights='imagenet', input_shape=input_size)
    else:
        print("Model not found, ensure model number is in range [0, 7].")
    
    if not finetune:
        model.trainable = False
    else:
        for layer in model.layers[:-tune_layers]:
            layer.trainable = False
    return model

In [13]:
def single_feature_model(num_classes, model_num=0, input_shape=(224,224,3), time_dist_shape=(None,224,224,3), dropout=0.5, dense_n=128, lstm_n=256, finetune=False, tune_layers=3):
    efficient_net = get_efficientnet(model_num=0, input_size=input_shape, finetune=finetune, tune_layers=tune_layers)
    
    # Transfer Layers
    input_tensor = Input(shape=time_dist_shape)
    efficient_layer = TimeDistributed(efficient_net, name=f"EfficientNetB{model_num}")(input_tensor)
    # Use pooling layer to reduce number of parameters by 12x
    efficient_out = TimeDistributed(GlobalAveragePooling2D())(efficient_layer)
#     efficient_out = TimeDistributed(Flatten())(efficient_layer)
    
    # LSTM Sequence Layer
    lstm = LSTM(lstm_n)(efficient_out)
    fc = Dense(dense_n, activation='relu')(lstm)
    fc_out = Dropout(dropout)(fc) 
    output = Dense(num_classes, activation='softmax')(fc_out)
    
    # Compile with Adam
    model = Model(input_tensor, output)
    model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model
    

In [14]:
# early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
# csv_callback = tf.keras.callbacks.CSVLogger('training.log')

def set_csv_callback(output_path, name: str):
    csv_path = Path(output_path, name)
    return tf.keras.callbacks.CSVLogger(csv_path)

def set_early_stop_callback(patience, monitor='val_loss'):
    return tf.keras.callbacks.EarlyStopping(monitor=monitor, patience=patience)

# RGB Only

## EfficientNetB0

In [15]:
rgb_model = single_feature_model(20, model_num=0)

In [16]:
rgb_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, 224, 224, 3 0         
_________________________________________________________________
EfficientNetB0 (TimeDistribu (None, None, 7, 7, 1280)  4049571   
_________________________________________________________________
time_distributed (TimeDistri (None, None, 1280)        0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               1573888   
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2580  

In [17]:
run_log = 'training_rgb_001.log'
run_model_summary = 'training_rgb_001_model_summary.txt'
model_checkpoint = 'training_rgb_001.best.hdf5'

with open(Path(OUTPUT_PATH, run_model_summary), 'w') as f:
    rgb_model.summary(print_fn=lambda x: f.write(x + '\n'))

early_stop_callback = set_early_stop_callback(5)
csv_callback = set_csv_callback(OUTPUT_PATH, run_log)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=Path(OUTPUT_PATH, model_checkpoint), 
                                                monitor='val_loss',
                                                verbose=0,
                                                save_best_only=True,
                                                mode='min')

In [18]:
# Train
rgb_train_history = rgb_model.fit(
    x=train_dataset, 
    validation_data=val_dataset, 
    epochs=100, 
    callbacks=[early_stop_callback, csv_callback, checkpoint], 
    workers=5, 
    use_multiprocessing=True
)

Epoch 1/100
307/307 [==============================] - 340s 1s/step - loss: 3.0044 - accuracy: 0.0650 - val_loss: 2.9039 - val_accuracy: 0.0969

Epoch 00001: val_loss improved from inf to 2.90392, saving model to D:\Uni\Honours\Project\data\autsl\outputs\runs\training_rgb_001.best.hdf5
Epoch 2/100
307/307 [==============================] - 304s 991ms/step - loss: 2.9180 - accuracy: 0.0775 - val_loss: 2.9015 - val_accuracy: 0.0689

Epoch 00002: val_loss improved from 2.90392 to 2.90149, saving model to D:\Uni\Honours\Project\data\autsl\outputs\runs\training_rgb_001.best.hdf5
Epoch 3/100
307/307 [==============================] - 301s 982ms/step - loss: 2.8993 - accuracy: 0.0804 - val_loss: 2.8742 - val_accuracy: 0.1020

Epoch 00003: val_loss improved from 2.90149 to 2.87422, saving model to D:\Uni\Honours\Project\data\autsl\outputs\runs\training_rgb_001.best.hdf5
Epoch 4/100
307/307 [==============================] - 300s 976ms/step - loss: 2.8210 - accuracy: 0.1205 - val_loss: 2.7205 -

## EfficientNetB2

In [15]:
rgb_model = single_feature_model(20, model_num=2)
rgb_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, 224, 224, 3 0         
_________________________________________________________________
EfficientNetB2 (TimeDistribu (None, None, 7, 7, 1280)  4049571   
_________________________________________________________________
time_distributed (TimeDistri (None, None, 1280)        0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               1573888   
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2580  

In [16]:
run_log = 'training_rgb_002.log'
run_model_summary = 'training_rgb_002_model_summary.txt'
model_checkpoint = 'training_rgb_002.best.hdf5'

with open(Path(OUTPUT_PATH, run_model_summary), 'w') as f:
    rgb_model.summary(print_fn=lambda x: f.write(x + '\n'))

early_stop_callback = set_early_stop_callback(5)
csv_callback = set_csv_callback(OUTPUT_PATH, run_log)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=Path(OUTPUT_PATH, model_checkpoint), 
                                                monitor='val_loss',
                                                verbose=0,
                                                save_best_only=True,
                                                mode='min')

In [17]:
# Train
rgb_train_history = rgb_model.fit(
    x=train_dataset, 
    validation_data=val_dataset, 
    epochs=100, 
    callbacks=[early_stop_callback, csv_callback, checkpoint], 
    workers=5, 
    use_multiprocessing=True
)

Epoch 1/100
307/307 [==============================] - 338s 1s/step - loss: 3.0098 - accuracy: 0.0770 - val_loss: 2.9129 - val_accuracy: 0.0867
Epoch 2/100
307/307 [==============================] - 316s 1s/step - loss: 2.9236 - accuracy: 0.1005 - val_loss: 2.8945 - val_accuracy: 0.0765
Epoch 3/100
307/307 [==============================] - 308s 1s/step - loss: 2.9074 - accuracy: 0.0863 - val_loss: 2.8724 - val_accuracy: 0.0816
Epoch 4/100
307/307 [==============================] - 318s 1s/step - loss: 2.8443 - accuracy: 0.0856 - val_loss: 2.7875 - val_accuracy: 0.1224
Epoch 5/100
307/307 [==============================] - 317s 1s/step - loss: 2.6833 - accuracy: 0.1352 - val_loss: 2.6702 - val_accuracy: 0.1531
Epoch 6/100
307/307 [==============================] - 307s 1s/step - loss: 2.4648 - accuracy: 0.1843 - val_loss: 2.5081 - val_accuracy: 0.1709
Epoch 7/100
307/307 [==============================] - 308s 1s/step - loss: 2.2574 - accuracy: 0.2224 - val_loss: 2.4625 - val_accuracy:

## EfficientNetB4

In [15]:
rgb_model = single_feature_model(20, model_num=4)
rgb_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, 224, 224, 3 0         
_________________________________________________________________
EfficientNetB4 (TimeDistribu (None, None, 7, 7, 1280)  4049571   
_________________________________________________________________
time_distributed (TimeDistri (None, None, 1280)        0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               1573888   
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2580  

In [16]:
run_log = 'training_rgb_003.csv'
run_model_summary = 'training_rgb_003_model_summary.txt'
model_checkpoint = 'training_rgb_003.best.hdf5'

with open(Path(OUTPUT_PATH, run_model_summary), 'w') as f:
    rgb_model.summary(print_fn=lambda x: f.write(x + '\n'))

early_stop_callback = set_early_stop_callback(5)
csv_callback = set_csv_callback(OUTPUT_PATH, run_log)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=Path(OUTPUT_PATH, model_checkpoint), 
                                                monitor='val_loss',
                                                verbose=0,
                                                save_best_only=True,
                                                mode='min')

In [17]:
# Train
rgb_train_history = rgb_model.fit(
    x=train_dataset, 
    validation_data=val_dataset, 
    epochs=100, 
    callbacks=[early_stop_callback, csv_callback, checkpoint], 
    workers=10, 
    use_multiprocessing=True
)

Epoch 1/100
307/307 [==============================] - 482s 2s/step - loss: 3.0219 - accuracy: 0.0688 - val_loss: 2.9300 - val_accuracy: 0.0791
Epoch 2/100
307/307 [==============================] - 301s 980ms/step - loss: 2.9701 - accuracy: 0.0726 - val_loss: 2.9170 - val_accuracy: 0.0714
Epoch 3/100
307/307 [==============================] - 300s 978ms/step - loss: 2.9455 - accuracy: 0.0725 - val_loss: 2.8932 - val_accuracy: 0.0842
Epoch 4/100
307/307 [==============================] - 300s 979ms/step - loss: 2.9112 - accuracy: 0.0799 - val_loss: 2.8667 - val_accuracy: 0.0867
Epoch 5/100
307/307 [==============================] - 300s 979ms/step - loss: 2.8223 - accuracy: 0.1002 - val_loss: 2.7662 - val_accuracy: 0.1097
Epoch 6/100
307/307 [==============================] - 300s 976ms/step - loss: 2.7099 - accuracy: 0.1126 - val_loss: 2.6568 - val_accuracy: 0.1378
Epoch 7/100
307/307 [==============================] - 299s 975ms/step - loss: 2.5711 - accuracy: 0.1438 - val_loss: 2.61

## EfficientNetB7

In [18]:
rgb_model = single_feature_model(20, model_num=7)
rgb_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None, 224, 224, 3 0         
_________________________________________________________________
EfficientNetB7 (TimeDistribu (None, None, 7, 7, 1280)  4049571   
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 1280)        0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               1573888   
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                2580

In [19]:
run_log = 'training_rgb_004.csv'
run_model_summary = 'training_rgb_004_model_summary.txt'
model_checkpoint = 'training_rgb_004.best.hdf5'

with open(Path(OUTPUT_PATH, run_model_summary), 'w') as f:
    rgb_model.summary(print_fn=lambda x: f.write(x + '\n'))

early_stop_callback = set_early_stop_callback(5)
csv_callback = set_csv_callback(OUTPUT_PATH, run_log)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=Path(OUTPUT_PATH, model_checkpoint), 
                                                monitor='val_loss',
                                                verbose=0,
                                                save_best_only=True,
                                                mode='min')

In [20]:
# Train
rgb_train_history = rgb_model.fit(
    x=train_dataset, 
    validation_data=val_dataset, 
    epochs=100, 
    callbacks=[early_stop_callback, csv_callback, checkpoint], 
    workers=10, 
    use_multiprocessing=True
)

Epoch 1/100
307/307 [==============================] - 312s 981ms/step - loss: 3.0273 - accuracy: 0.0694 - val_loss: 2.9340 - val_accuracy: 0.0816
Epoch 2/100
307/307 [==============================] - 296s 964ms/step - loss: 2.9353 - accuracy: 0.0721 - val_loss: 2.9268 - val_accuracy: 0.0791
Epoch 3/100
307/307 [==============================] - 297s 968ms/step - loss: 2.9195 - accuracy: 0.0788 - val_loss: 2.8691 - val_accuracy: 0.0918
Epoch 4/100
307/307 [==============================] - 299s 974ms/step - loss: 2.8653 - accuracy: 0.0977 - val_loss: 2.8018 - val_accuracy: 0.1020
Epoch 5/100
307/307 [==============================] - 295s 960ms/step - loss: 2.7413 - accuracy: 0.1293 - val_loss: 2.7500 - val_accuracy: 0.1276
Epoch 6/100
307/307 [==============================] - 295s 961ms/step - loss: 2.6290 - accuracy: 0.1377 - val_loss: 2.6529 - val_accuracy: 0.1531
Epoch 7/100
307/307 [==============================] - 295s 961ms/step - loss: 2.4785 - accuracy: 0.1809 - val_loss: 2

## Run more!

In [23]:
NUM_TRIALS = 2

In [24]:
# B0
TRIAL_START = 7
for i in range(NUM_TRIALS):
    trial_num = i + TRIAL_START
    rgb_model = single_feature_model(20, model_num=0)
    
    run_log = f'training_rgb_{trial_num:03}.csv'
    run_model_summary = f'training_rgb_{trial_num:03}_model_summary.txt'
    model_checkpoint = f'training_rgb_{trial_num:03}.best.hdf5'

    with open(Path(OUTPUT_PATH, run_model_summary), 'w') as f:
        rgb_model.summary(print_fn=lambda x: f.write(x + '\n'))

    early_stop_callback = set_early_stop_callback(5)
    csv_callback = set_csv_callback(OUTPUT_PATH, run_log)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=Path(OUTPUT_PATH, model_checkpoint), 
                                                    monitor='val_loss',
                                                    verbose=0,
                                                    save_best_only=True,
                                                    mode='min')
    
    # Train
    rgb_train_history = rgb_model.fit(
        x=train_dataset, 
        validation_data=val_dataset, 
        epochs=100, 
        callbacks=[early_stop_callback, csv_callback, checkpoint], 
        workers=10, 
        use_multiprocessing=True
    )

Epoch 1/100
307/307 [==============================] - 1504s 4s/step - loss: 3.0251 - accuracy: 0.0701 - val_loss: 2.9292 - val_accuracy: 0.0842
Epoch 2/100
307/307 [==============================] - 1205s 4s/step - loss: 2.9458 - accuracy: 0.0792 - val_loss: 2.9385 - val_accuracy: 0.0791
Epoch 3/100
307/307 [==============================] - 1051s 3s/step - loss: 2.9160 - accuracy: 0.0826 - val_loss: 2.8985 - val_accuracy: 0.0944
Epoch 4/100
307/307 [==============================] - 1077s 4s/step - loss: 2.8777 - accuracy: 0.0845 - val_loss: 2.8385 - val_accuracy: 0.1046
Epoch 5/100
307/307 [==============================] - 1060s 3s/step - loss: 2.8326 - accuracy: 0.0938 - val_loss: 2.7681 - val_accuracy: 0.1173
Epoch 6/100
307/307 [==============================] - 1047s 3s/step - loss: 2.7126 - accuracy: 0.1298 - val_loss: 2.6974 - val_accuracy: 0.1224
Epoch 7/100
307/307 [==============================] - 1050s 3s/step - loss: 2.5721 - accuracy: 0.1337 - val_loss: 2.5618 - val_ac

KeyboardInterrupt: 

In [ ]:
# B2
TRIAL_START += NUM_TRIALS
for i in range(NUM_TRIALS):
    trial_num = i + TRIAL_START
    rgb_model = single_feature_model(20, model_num=2)
    
    run_log = f'training_rgb_{trial_num:03}.csv'
    run_model_summary = f'training_rgb_{trial_num:03}_model_summary.txt'
    model_checkpoint = f'training_rgb_{trial_num:03}.best.hdf5'

    with open(Path(OUTPUT_PATH, run_model_summary), 'w') as f:
        rgb_model.summary(print_fn=lambda x: f.write(x + '\n'))

    early_stop_callback = set_early_stop_callback(5)
    csv_callback = set_csv_callback(OUTPUT_PATH, run_log)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=Path(OUTPUT_PATH, model_checkpoint), 
                                                    monitor='val_loss',
                                                    verbose=0,
                                                    save_best_only=True,
                                                    mode='min')
    
    # Train
    rgb_train_history = rgb_model.fit(
        x=train_dataset, 
        validation_data=val_dataset, 
        epochs=100, 
        callbacks=[early_stop_callback, csv_callback, checkpoint], 
        workers=10, 
        use_multiprocessing=True
    )

In [ ]:
# B4
TRIAL_START += NUM_TRIALS
for i in range(NUM_TRIALS):
    trial_num = i + TRIAL_START
    rgb_model = single_feature_model(20, model_num=4)
    
    run_log = f'training_rgb_{trial_num:03}.csv'
    run_model_summary = f'training_rgb_{trial_num:03}_model_summary.txt'
    model_checkpoint = f'training_rgb_{trial_num:03}.best.hdf5'

    with open(Path(OUTPUT_PATH, run_model_summary), 'w') as f:
        rgb_model.summary(print_fn=lambda x: f.write(x + '\n'))

    early_stop_callback = set_early_stop_callback(5)
    csv_callback = set_csv_callback(OUTPUT_PATH, run_log)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=Path(OUTPUT_PATH, model_checkpoint), 
                                                    monitor='val_loss',
                                                    verbose=0,
                                                    save_best_only=True,
                                                    mode='min')
    
    # Train
    rgb_train_history = rgb_model.fit(
        x=train_dataset, 
        validation_data=val_dataset, 
        epochs=100, 
        callbacks=[early_stop_callback, csv_callback, checkpoint], 
        workers=10, 
        use_multiprocessing=True
    )

In [ ]:
# B7
TRIAL_START += NUM_TRIALS
for i in range(NUM_TRIALS):
    trial_num = i + TRIAL_START
    rgb_model = single_feature_model(20, model_num=7)
    
    run_log = f'training_rgb_{trial_num:03}.csv'
    run_model_summary = f'training_rgb_{trial_num:03}_model_summary.txt'
    model_checkpoint = f'training_rgb_{trial_num:03}.best.hdf5'

    with open(Path(OUTPUT_PATH, run_model_summary), 'w') as f:
        rgb_model.summary(print_fn=lambda x: f.write(x + '\n'))

    early_stop_callback = set_early_stop_callback(5)
    csv_callback = set_csv_callback(OUTPUT_PATH, run_log)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=Path(OUTPUT_PATH, model_checkpoint), 
                                                    monitor='val_loss',
                                                    verbose=0,
                                                    save_best_only=True,
                                                    mode='min')
    
    # Train
    rgb_train_history = rgb_model.fit(
        x=train_dataset, 
        validation_data=val_dataset, 
        epochs=100, 
        callbacks=[early_stop_callback, csv_callback, checkpoint], 
        workers=10, 
        use_multiprocessing=True
    )